# Pré-processamento dos Dados

> Link para acessar o conjunto de dados: [Medical Personal Protective Equipment Images](https://www.kaggle.com/datasets/skaarface/medical-personal-protective-equipment-images)

O ```CPPE - 5``` ("*Medical Personal Protective Equipment Images*") é um novo conjunto de dados com o objetivo de permitir o estudo da categorização por imagens de equipamentos de proteção individual médico, o que não é possível com outros conjuntos de dados populares que se concentram em categorias de nível amplo.

### Importação dos Pacotes Utilitários

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import shutil
import json
import cv2
import os

### Organização das Pastas dos Arquivos

In [20]:
# criando pastas para organizar o conjunto de dados no formato aceito pela YOLO
shutil.unpack_archive('data.zip')
os.mkdir('formatted-dataset')
os.mkdir('formatted-dataset/images')
os.mkdir('formatted-dataset/images/train')
os.mkdir('formatted-dataset/images/val')
os.mkdir('formatted-dataset/labels')
os.mkdir('formatted-dataset/labels/train')
os.mkdir('formatted-dataset/labels/val')

In [21]:
# movendo as imagens do conjunto de dados para as novas pastas criadas
for i in range(0, 1029):
    if (i + 1) <= 1000:
        shutil.move(src = f'Dataset/images/{i+1}.png', dst = 'formatted-dataset/images/train') 
    else:
        shutil.move(src = f'Dataset/images/{i+1}.png', dst = 'formatted-dataset/images/val')

### Organizando e Pré-processando a estrutura dos Dados

In [ ]:
# carregando os arquivos JSON com as informações e rotulações da base de dados
with open('Dataset/annotations/train.json') as json_file:
    train = json.load(json_file)
with open('Dataset/annotations/test.json') as json_file:
    test = json.load(json_file)

In [ ]:
def normalize_bbox(dict_ref, index):
    '''
    normaliza os valores das coordenadas de caixas delimitadoras
    
    Args:
        dict_ref (dict) --> dicionário com as informações da base de dados
        index (int) --> valor do índice do dicionário a ser iterado 
    '''
    
    image_id = dict_ref['annotations'][index]['image_id']
    if image_id > 1000: image_id -= 1000

    #padrão (x_min, y_min, width, height) -> (x_center, y_center, width/2, height/2)
    x_center = dict_ref['annotations'][index]['bbox'][0] + dict_ref['annotations'][index]['bbox'][2]/2
    y_center = dict_ref['annotations'][index]['bbox'][1] + dict_ref['annotations'][index]['bbox'][3]/2
    dict_ref['annotations'][index]['bbox'][0] = x_center
    dict_ref['annotations'][index]['bbox'][1] = y_center
 
    dict_ref['annotations'][index]['bbox'][0] /= dict_ref['images'][image_id - 1]['width']
    dict_ref['annotations'][index]['bbox'][1] /= dict_ref['images'][image_id - 1]['height']
    dict_ref['annotations'][index]['bbox'][2] /= dict_ref['images'][image_id - 1]['width']
    dict_ref['annotations'][index]['bbox'][3] /= dict_ref['images'][image_id - 1]['height']

In [22]:
# normalizando as caixas delimitadoras para o padrão aceito pela YOLO
for i in range(0, len(train['annotations'])):
    normalize_bbox(dict_ref = train, index = i)
for i in range(0, len(test['annotations'])):
    normalize_bbox(dict_ref = test, index = i)

In [23]:
def make_labels(dict_ref, path_dir):
    '''
    cria arquivos txt com os rótulos para a rede de detecção

    Args:
        dict_ref (dict) --> dicionário com as imformações do conjunt0 de dados
    '''

    for i in range(0, len(dict_ref['annotations'])):
        value = str(dict_ref['annotations'][i]['category_id'] - 1) + ' '
        value += str(dict_ref['annotations'][i]['bbox'][0]) + ' ' 
        value += str(dict_ref['annotations'][i]['bbox'][1]) + ' '
        value += str(dict_ref['annotations'][i]['bbox'][2]) + ' '
        value += str(dict_ref['annotations'][i]['bbox'][3])

        image_id = dict_ref['annotations'][i]['image_id']
        with open(f'{path_dir}{image_id}.txt', 'a') as f:
            f.write(value + '\n')

In [ ]:
# criando arquivos txt com as coordenadas das caixas delimitadoras das imagens
make_labels(dict_ref = train, path_dir = 'formatted-dataset/labels/train/')
make_labels(dict_ref = test, path_dir = 'formatted-dataset/labels/val/')